In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import plotly
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import seaborn as sb

import datetime

# Import data

In [2]:
df = pd.read_csv('data_raw/godaddy-microbusiness-density-forecasting/train.csv')
df_test = pd.read_csv('data_raw/godaddy-microbusiness-density-forecasting/test.csv')
df_census = pd.read_csv('data_raw/godaddy-microbusiness-density-forecasting/census_starter.csv')

In [3]:
df.shape, df_test.shape

((122265, 7), (25080, 3))

# Get statistical analysis about columns in Train

In [4]:
df.head()

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243


In [5]:
df.tail()

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100
122264,56045_2022-10-01,56045,Weston County,Wyoming,2022-10-01,1.785395,100


In [6]:
df.describe()

,cfips,microbusiness_density,active
count,122265.000000,122265.000000,1.222650e+05
mean,30376.037640,3.817671,6.442858e+03
std,15143.508721,4.991087,3.304001e+04
min,1001.000000,0.000000,0.000000e+00
25%,18177.000000,1.639344,1.450000e+02
50%,29173.000000,2.586543,4.880000e+02
75%,45077.000000,4.519231,2.124000e+03
max,56045.000000,284.340030,1.167744e+06


In [7]:
# Convert to datetime
df['first_day_of_month'] = pd.to_datetime(df['first_day_of_month']).dt.date

In [8]:
df['state'].nunique()

51

In [9]:
df['county'].nunique()

1871

In [10]:
df.groupby('state')['county'].nunique().sort_values(ascending=False)

state
Texas                   254
Georgia                 159
Virginia                130
Kentucky                120
Missouri                115
Kansas                  105
Illinois                102
North Carolina          100
Iowa                     99
Tennessee                95
Nebraska                 92
Indiana                  92
Ohio                     88
Minnesota                87
Michigan                 83
Mississippi              82
Oklahoma                 77
Arkansas                 75
Wisconsin                72
Pennsylvania             67
Alabama                  67
Florida                  67
South Dakota             65
Colorado                 64
Louisiana                64
New York                 62
California               58
Montana                  56
West Virginia            55
North Dakota             53
South Carolina           46
Idaho                    44
Washington               39
Oregon                   36
New Mexico               33
Utah          

### Restructure df to timeseries

In [11]:
df_time_series = df.pivot_table(
    index=['cfips', 'state', 'county'],
    columns='first_day_of_month',
    values='microbusiness_density'
).reset_index([1, 2])

df_time_series.head()

first_day_of_month,state,county,2019-08-01,2019-09-01,2019-10-01,2019-11-01,2019-12-01,2020-01-01,2020-02-01,2020-03-01,...,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01
cfips,,,,,,,,,,,,,,,,,,,,,
1001,Alabama,Autauga County,3.007682,2.884870,3.055843,2.993233,2.993233,2.969090,2.909326,2.933231,...,3.296781,3.334431,3.336785,3.372082,3.313253,3.346197,3.437971,3.423852,3.442677,3.463856
1003,Alabama,Baldwin County,7.239156,7.290936,7.425439,7.426071,7.470274,7.413655,7.282522,7.309610,...,7.733397,7.823300,7.945311,7.979170,7.907365,8.507496,8.573463,8.491150,8.341701,8.359798
1005,Alabama,Barbour County,1.073138,0.995794,1.160149,1.000628,1.000628,1.027229,1.022314,1.032144,...,1.186629,1.206827,1.196728,1.191678,1.186629,1.191678,1.216926,1.196728,1.206827,1.232074
1007,Alabama,Bibb County,1.310777,1.305176,1.254761,1.254761,1.265965,1.253638,1.248041,1.264831,...,1.214165,1.236650,1.264755,1.253513,1.247892,1.275998,1.326588,1.292861,1.315346,1.287240
1009,Alabama,Blount County,1.544148,1.575892,1.546415,1.573625,1.555485,1.573217,1.536999,1.541526,...,1.752923,1.777708,1.797986,1.764189,1.748417,1.773202,1.831783,1.836289,1.852060,1.831783


# Plot some graphs

In [ ]:
%matplotlib inline
states = set(df['state'].to_list())
for state in states:
    df_state = df[df['state'] == state]
    df_state.plot(x="first_day_of_month", y="microbusiness_density", title=state)

In [12]:
fig = ff.create_choropleth(
    fips = df_time_series.index.to_list(),
    values = df_time_series.iloc[:, -1].to_list(),
    mapbox_style = 'carto-positron',
    show_hover=True,
    binning_endpoints = list(np.arange(0, 10, 0.25)),
    width = 1000,
    height = 600
)
fig.update_layout(showlegend=True, title='Microbusinees Density in USD by county.')
fig.show()

ImportError: geopandas, pyshp and shapely must be installed for this figure factory.

Run the following commands to install the correct versions of the following modules:

```
$ pip install geopandas==0.3.0
$ pip install pyshp==1.2.10
$ pip install shapely==1.6.3
```
If you are using Windows, follow this post to properly install geopandas and dependencies:http://geoffboeing.com/2014/09/using-geopandas-windows/

If you are using Anaconda, do not use PIP to install the packages above. Instead use conda to install them:

```
$ conda install plotly
$ conda install geopandas
```

# Process test data

In [5]:
df_test = df_test.merge(df[['cfips', 'county', 'state']].drop_duplicates(), how='left')

In [6]:
df_test.head()

,row_id,cfips,first_day_of_month,county,state
0,1001_2022-11-01,1001,2022-11-01,Autauga County,Alabama
1,1003_2022-11-01,1003,2022-11-01,Baldwin County,Alabama
2,1005_2022-11-01,1005,2022-11-01,Barbour County,Alabama
3,1007_2022-11-01,1007,2022-11-01,Bibb County,Alabama
4,1009_2022-11-01,1009,2022-11-01,Blount County,Alabama
